In [1]:
from bs4 import BeautifulSoup, Tag, NavigableString
import requests

In [2]:
job_url = 'https://www.linkedin.com/jobs/search?keywords=Machine%2BLearning&location=United%2BStates&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&currentJobId=4206679802&position=2&pageNum=0'
response = requests.get(job_url)
html = response.text
soup = BeautifulSoup(html, "html.parser")
title_element = soup.select_one("h1")
title = title_element.get_text().strip()

company_element = soup.select_one("[data-tracking-control-name=\"public_jobs_topcard-org-name\"]")
company_name = company_element.get_text().strip()
company_url = company_element["href"]

location_element = soup.select_one(".topcard__flavor--bullet")
location = location_element.get_text().strip()

applicants_element = soup.select_one(".num-applicants__caption")
applicants = applicants_element.get_text().strip()
salary_element = soup.select_one(".salary")
if salary_element is not None:
    salary = salary_element.get_text().strip()
else:
    salary = None

print(location)
print(applicants)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [3]:
description_element = soup.select_one(".description__text .show-more-less-html")
markup = description_element.select_one(".show-more-less-html__markup")
print(markup)


<div class="show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden">
<strong>Location:</strong> Hybrid (some travel to Washington, DC area)<br/><br/><strong>Work Type</strong>: Full-Time<br/><br/><strong><strong>About PingWind<br/><br/></strong></strong>PingWind is an SBA-certified Service-Disabled Veteran Owned Small Business (SDVOSB) delivering mission-critical technology solutions to federal government partners. We help clients secure information, modernize systems, and optimize performance through advanced technologies—including AI-driven operations and machine learning. Our team values innovation, teamwork, and a commitment to making a meaningful difference in the world. Join us to work with smart, dedicated individuals on important missions that shape the future.<br/><br/><strong><strong>Position Summary<br/><br/></strong></strong>We are seeking a highly motivated Junior AI/ML Engineer to join our growing team. In this role, you will contr

In [4]:
data = {}
# 1) find every <strong> whose parent is NOT another <strong>
headers = [h for h in soup.find_all("strong") if h.parent.name != "strong"]

for head in headers:
    # normalize label text
    label = head.get_text(strip=True).rstrip(":")
    parts = []
    # 2) walk siblings until the next top-level <strong>
    for sib in head.next_siblings:
        # stop at next header
        if isinstance(sib, Tag) and sib.name == "strong" and sib.parent.name != "strong":
            break
        # skip pure <br> spacing
        if isinstance(sib, Tag) and sib.name == "br":
            continue
        # text nodes
        if isinstance(sib, NavigableString):
            txt = sib.strip()
            if txt:
                parts.append(txt)
        # any other tag (e.g. <ul>, <p>)
        elif isinstance(sib, Tag):
            txt = sib.get_text(" ", strip=True)
            if txt:
                parts.append(txt)
    data[label] = " ".join(parts).strip()

required_qualification = data['Required Qualifications']
preferred_qualification = data['Preferred Qualifications']

In [5]:
criteria = []
criteria_elements = soup.select(".description__job-criteria-list li")
for criteria_element in criteria_elements:
    print(criteria_element)
    name_element = criteria_element.select_one(".description__job-criteria-subheader")
    name = name_element.get_text().strip()

    value_element = criteria_element.select_one(".description__job-criteria-text")
    value = value_element.get_text().strip()

    criteria.append({
        "name": name,
        "value": value
    })

print(criteria)

<li class="description__job-criteria-item">
<h3 class="description__job-criteria-subheader">
            Seniority level
          </h3>
<span class="description__job-criteria-text description__job-criteria-text--criteria">
            Not Applicable
          </span>
</li>
<li class="description__job-criteria-item">
<h3 class="description__job-criteria-subheader">
            Employment type
          </h3>
<span class="description__job-criteria-text description__job-criteria-text--criteria">
            Internship
          </span>
</li>
<li class="description__job-criteria-item">
<h3 class="description__job-criteria-subheader">
              Job function
            </h3>
<span class="description__job-criteria-text description__job-criteria-text--criteria">
              Engineering and Information Technology
            </span>
</li>
<li class="description__job-criteria-item">
<h3 class="description__job-criteria-subheader">
              Industries
            </h3>
<span class="d

In [6]:
job = {
    "url": job_url,
    "title": title,
    "company": {
        "name": company_name,
        "url": company_url
    },
    "location": location,
    "applications": applicants,
    "salary": salary,
    "required qualification": required_qualification,
    "preferred qualification": preferred_qualification,
    "criteria": criteria
}

In [7]:
job

{'url': 'https://www.linkedin.com/jobs/view/junior-artificial-intelligence-ai-machine-learning-ml-engineer-at-pingwind-inc-sdvosb-4211917486?position=1&pageNum=0&refId=tpce3jhwfSQDn1S%2B1BsVrg%3D%3D&trackingId=q98vLda4U3aFybBlVxCnpQ%3D%3D',
 'title': 'Junior Artificial Intelligence (AI) / Machine Learning (ML) Engineer',
 'company': {'name': 'PingWind Inc. (SDVOSB)',
  'url': 'https://www.linkedin.com/company/pingwind-inc-?trk=public_jobs_topcard-org-name'},
 'location': 'Washington DC-Baltimore Area',
 'applications': 'Be among the first 25 applicants',
 'salary': None,
 'required qualification': 'Bachelor’s degree in Computer Science, Data Science, Engineering, Mathematics, or a related field. Or 2 years work experience in software engineering with some AI/ML practices Proficiency in Python and familiarity with ML libraries such as TensorFlow, PyTorch, Scikit-learn, or Keras Strong understanding of linear algebra, probability, statistics, and optimization techniques Experience with d